In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator
import random
import numpy as np
import pandas as pd
from typing import List, Dict, Union
import yaml
from bisect import bisect_left
from tqdm import tqdm
import calendar
import os
import math
from PIL import Image
import cv2
# from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration, AutoConfig
import torch
from transformers import AutoProcessor, Pix2StructForConditionalGeneration
import gc
import warnings
warnings.filterwarnings("ignore")

2023-07-05 02:21:06.409225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-07-05 02:21:06.409378: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/li

In [2]:
chart_type = "line"
version = 11

ver1.1 : gtの小数点を丸め処理

In [3]:
save_dir = f"./../input/benetech-synthesis-4th/{chart_type}_images_ver{version}"
os.makedirs(save_dir, exist_ok=True)
processor = AutoProcessor.from_pretrained("google/deplot")

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

graph_type  
title  
x_label  
y_label  
points  
x  
y  
color  
line_stype  
line_marker  
line_width  
background_color  
size  


In [5]:
def round_numerical(value: Union[int, float, str]) -> Union[str, float]:
    if isinstance(value, float):
        value = "{:.8f}".format(value)

        if "." in value:
            integer, decimal = value.split(".")
        else:
            integer = value
            
        if abs(float(integer)) >= 1000:
            integer = integer[:3] + "0"*(len(integer)-3)
            return integer
        elif abs(float(integer)) < 1000 and abs(float(integer)) >= 100:
            return integer
        elif abs(float(integer)) < 100 and abs(float(integer)) >= 10:
            decimal = decimal[:1]
        elif abs(float(integer)) < 10 and abs(float(integer)) >= 1:
            decimal = decimal[:2]
        else:
            decimal = decimal[:4]

        value = integer + "." + decimal
        
    if  isinstance(value, int):
        value = str(value)
        if abs(int(value)) >= 1000:
            value = value[:3] + "0"*(len(value)-3)
            return value        
        
    return value

def round_float(value: Union[int, float, str]) -> Union[str, float]:
    if isinstance(value, float):
        value = "{:.8f}".format(value)

        if "." in value:
            integer, decimal = value.split(".")
            if abs(float(value)) < 0.001:
                decimal = decimal[:6]
            elif abs(float(value)) < 0.01:
                decimal = decimal[:5]
            elif abs(float(integer)) >= 100:
                return integer
            elif abs(float(integer)) < 100 and abs(float(integer)) >= 10:
                decimal = decimal[:1]
            elif abs(float(integer)) < 10 and abs(float(integer)) >= 1:
                decimal = decimal[:2]
            else:
                decimal = decimal[:4]

            value = integer + "." + decimal
    return value

In [6]:
def get_x_values(p):

    r = np.random.rand()
    
    # 0付近小数
    if r < 0.5:
        a = random.uniform(-5, 3)
        b = random.uniform(a+1, 5)
        
    # 0 ~ 10小数
    else:
        a = random.uniform(0, 3)
        b = random.uniform(5, 10)
    
    return np.linspace(a, b, p)

In [7]:
def get_x_values_after(p):

    r = np.random.rand()
    
    # 西暦
    if r < 0.15:
        a = random.randint(1900, 2020)
        b = random.uniform(a+10, 2040)
    # 小数点
    elif r < 0.3:
        a = 0
        b = random.uniform(0.2, 1)
    # 1000（0固定）
    elif r < 0.45:
        a = 0
        b = random.randint(10, 1000)
    # 1000まで
    elif r < 0.6:
        a = random.randint(10, 990)
        b = random.randint(a+10, 1000)
    # 1000以上（0固定）
    elif r < 0.75:
        a = 0
        b = random.randint(10, 9900)
    # 1000以上
    elif r < 0.9:
        a = random.randint(10, 1000)
        b = random.randint(a+10, 9900)
    # 0付近
    else:
        a = random.uniform(-3, 0)
        b = random.uniform(0, 3)
        
    # ランダムでマイナスにする
    if  np.random.rand() < 0.2:
        a, b = -b, -a
        
    return np.linspace(a, b, p)

In [8]:
def get_y_values(x):
    
    # オフセット
    r = np.random.rand()
    if r < 0.5:
        offset = random.uniform(0, 5)
    else:
        offset = 0
    
    # ランダムノイズ
    r = np.random.rand()
    if r < 0.2:
        h = random.uniform(0, max(x)//2)
        noise = np.random.normal(0, 0.1, len(x))
    else:
        noise = np.random.normal(0, 0, len(x))
    
    r = np.random.rand()
    # sin
    if r < 0.05:
        y = np.sin(x)
    elif r < 0.1:
        y = np.cos(x)
    elif r < 0.2:
        y = np.sqrt(np.abs(x))
    elif r < 0.3:
        e = math.e
        y = 1 / (1 + e**-x)
    elif r < 0.4:
        y = np.sinh(x)
    elif r < 0.5:
        m = random.choice(x)
        s = random.uniform(1, 3)
        y = np.exp(-(x-m)**2 / (2*s*s)) / np.sqrt(2 * np.pi)
    elif r < 0.6:
        y = np.exp(x)
    elif r < 0.65:
        y = x
    elif r < 0.7:
        y = -x
    elif r < 0.8:
        m = random.choice(x)
        y = (x-m)**2    
    else:
        a1 = random.uniform(0, 30)
        a2 = random.uniform(0, 30)
        a3 = random.uniform(0, 30)
        a4 = random.uniform(0, 30)
        a5 = random.uniform(0, 30)
        y = a1*x+a2*np.power(x, 2) + a2*np.power(x, 2) + a3*np.power(x, 3) + a4*np.power(x, 4) + a5*np.power(x, 5)
    return y + offset + noise

In [9]:
def get_x_label(source):
    
    if source == "None":
        return ""
    
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [10]:
def get_y_label(source):
    
    if source == "None":
        return ""
    
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [11]:
def get_title(source):
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [12]:
def get_font(source):
    val = []
    with open(f"{source}", "r") as f:
        for line in f:
            val.append(line.strip())
    return random.choice(val)

In [13]:
with open('graph_generation_ver4.yaml') as file:
    data_gen = yaml.safe_load(file)

In [14]:
def generate_graph(data, idx):
    # データ点数
    num_points = random.randint(50, 1000)
    
    # データ作成
    x_label_source = random.choices(data["numerical_label"]["value"], k=1, weights=data["numerical_label"]["weight"])[0]
    x_label = get_x_label(x_label_source)
    #print(x_label)
    y_label_source = random.choices(data["numerical_label"]["value"], k=1, weights=data["numerical_label"]["weight"])[0]
    y_label = get_y_label(y_label_source)
    #print(y_label)
    x_values = get_x_values(num_points)
    #print(x_values)
    y_values = get_y_values(x_values)
    #print(y_values)
    
    while max(y_values) > 100000:
        y_values = [y/10 for y in y_values]
    
    rand = np.random.rand()
    if rand < 0.2 and max(y_values) < 1:
        y_values = [y/1000 for y in y_values]
        
    while max([abs(yi) for yi in y_values]) < 1e-5:
        y_values = [y*10 for y in y_values]
        
    
    # x_valuesを差し替え
    if np.random.rand() < 0.8:
        x_values = get_x_values_after(num_points)
        
        
    # フォントとフォントサイズの設定
    fontsize = random.choices(data["fontsize"]["value"], k=1, weights=data["fontsize"]["weight"])[0]
    plt.rcParams['font.size'] = fontsize
    
    font_source = random.choices(data["font"]["value"], k=1, weights=data["font"]["weight"])[0]
    font = get_font(font_source)
    #print(font)
    
    # ライングラフの見た目を設定
    color = random.choices(data["colors"]["value"], k=1, weights=data["colors"]["weight"])[0]
    linewidth = random.choices(data["linewidth"]["value"], k=1, weights=data["linewidth"]["weight"])[0]
    linestyle = random.choices(data["linestyle"]["value"], k=1, weights=data["linestyle"]["weight"])[0]
    marker = random.choices(data["marker"]["value"], k=1, weights=data["marker"]["weight"])[0]
    marker_size = random.choices(data["marker_size"]["value"], k=1, weights=data["marker_size"]["weight"])[0]
    if num_points > 21:
        marker = None
    # scatterと紛らわしいので置き換え
    if linestyle == ":":
        linestyle = "-"
    
    
    # グラフのサイズを指定 例：4.56=456 pixel
    size1 = random.randint(400, 800)
    size2 = random.randint(300, size1)
    size1 /= 100
    size2 /= 100
    
    facecolor =  random.choices(data["facecolor"]["value"], k=1, weights=data["facecolor"]["weight"])[0]
    
    fig, ax = plt.subplots(figsize=(size1, size2))
    fig.set_facecolor(facecolor)
    
    
    ax.plot(x_values, y_values, color=color, linewidth=linewidth, linestyle=linestyle, marker=marker, markersize=marker_size)
    #ax.set_xlabel(x_label)
    
    # major tickの設定
    tick_direction = random.choices(data["tick_direction"]["value"], k=1, weights=data["tick_direction"]["weight"])[0]
    # 削除
    rand = np.random.rand()
    if rand < 0.2:
        plt.tick_params(axis='both', which='major', length=0)
    # major
    elif rand < 0.6:
        plt.tick_params(axis='both', which='major', direction=tick_direction)
    # major and minor
    else:
        plt.tick_params(axis='both', which='both', direction=tick_direction)
        plt.minorticks_on()
        
    # mirror tickの表示、topとrightのspine設定
    rand = np.random.rand()
    # 両軸表示
    if rand < 0.2:
        ax.yaxis.set_ticks_position('both')
        ax.xaxis.set_ticks_position('both')
    # 上部と右側の枠線を非表示にする
    elif rand < 0.4:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
    #  x-tickにフォント設定
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)

    #  y-tickにフォント設定
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
    
    # ラベルにフォント設定
    ax.set_xlabel(x_label, fontname=font, labelpad=20)
    ax.set_ylabel(y_label, fontname=font, labelpad=20)
    
    # グラフタイトル
    title_source = random.choices(data["title"]["value"], k=1, weights=data["title"]["weight"])[0]
    if title_source != "None":
        title = get_title(title_source)
        ax.set_title(title, fontname=font)
    else:
        title = ""
    
    # 背景色
    if color != "k":
        background_color = random.choices(data["background_color"]["value"], k=1, weights=data["background_color"]["weight"])[0]
        ax.set_facecolor(background_color)
        
    # grid線
    grid = random.choices(data["grid"]["value"], k=1, weights=data["grid"]["weight"])[0]
    if grid != "None":
        grid_color = background_color = random.choices(data["background_color"]["value"], k=1, weights=data["background_color"]["weight"])[0]
        grid_linestyle = random.choices(data["linestyle"]["value"], k=1, weights=data["linestyle"]["weight"])[0]
        if grid == "bose":
            ax.grid(color=grid_color, linestyle=grid_linestyle)
        else:
            ax.grid(color=grid_color, linestyle=grid_linestyle, axis=grid)
    
    
    # x軸の拡張
    x_range = max(x_values)-min(x_values)
    xticks = ax.get_xticks()
    min_x = min(x_values)
    max_x = max(x_values)
    # 両サイド拡張
    if np.random.rand() < 0.3:
        x_scale = random.uniform(0, 1)
        # 両サイド
        if np.random.rand() < 0.5:
            plt.xlim(min(x_values)-x_range*x_scale, max(x_values)+x_range*x_scale)
        # 左端
        elif np.random.rand() < 0.75:
            plt.xlim(min(x_values)-x_range*x_scale, max(x_values))
        # 右端
        else:
            plt.xlim(min(x_values), max(x_values)+x_range*x_scale)
        
    # 片方タイト
    elif np.random.rand() < 0.5:
        plt.xlim(xticks[1], max(xticks))
        min_x = xticks[1]
    elif np.random.rand() < 0.7:
        plt.xlim(min(xticks), xticks[-2])
        max_x = xticks[-2]
    # 両サイドタイト
    elif np.random.rand() < 0.9:
        plt.xlim(xticks[1], xticks[-2])
        min_x = xticks[1]
        max_x = xticks[-2]
        
    # y軸の拡張
    y_range = max(y_values)-min(y_values)
    yticks = ax.get_yticks()
    min_y = yticks[0]
    max_y = yticks[-1]
    # 両サイド拡張
    if np.random.rand() < 0.2:
        y_scale = random.uniform(0, 1)
        plt.ylim(min(y_values)-y_range*y_scale, max(y_values)+y_range*y_scale)
    elif np.random.rand() < 0.2:
        plt.ylim(yticks[1], max(yticks))
        min_y = yticks[1]
    elif np.random.rand() < 0.2:
        plt.ylim(min(yticks), yticks[-2])
        max_y = yticks[-2]
    elif np.random.rand() < 0.8:
        plt.ylim(yticks[1], yticks[-2])
        min_y = yticks[1]
        max_y = yticks[-2]

    #  x, yのgt作成
    xticks = ax.get_xticks()
    
    gt_x = []
    gt_y = []

    # tick上に値が存在するものだけを選別
    # tick間隔の3%のずれはtick上にポイントが存在するとみなす（目視でもわかりにくいので）
    x_eps = (xticks[1] - xticks[0]) * 0.03
    y_eps = (yticks[1] - yticks[0]) * 0.03
    for x in xticks:
        if (x >= min_x-x_eps and x <= max_x+x_eps) & (x >= min(x_values)-x_eps and x <= max(x_values)+x_eps):
            ix = bisect_left(x_values, x)
            yi = y_values[max(0, ix-1)]
            if yi >= min_y-y_eps and yi <= max_y+y_eps:
                gt_x.append(x)
                gt_y.append(yi)
    
    # 画像の保存
    id = str(idx).zfill(8)
    path = os.path.join(save_dir, f"{id}.jpg")
    fig.savefig(path, bbox_inches='tight')
    
    # 画質を落とす
    image = Image.open(path)
    quality = random.randint(5, 100)
    # greyscaleで保存
    if np.random.rand() < 0.05:
        image = image.convert("L")
        image = image.convert("RGB")
    image.save(path, quality=quality)
    
    # 整数の場合は小数点削除
    if np.all([x.is_integer() for x in gt_x]):
        gt_x = [int(x) for x in gt_x]
    else:
        gt_x = [float(x) for x in gt_x]
        # 誤差を丸める
        gt_x = [int((x+0.00001)*10000)//1/10000  if x > 0 else int((x-0.00001)*10000)//1/10000 for x in gt_x]
    
    gt_y = [round_float(y) for y in gt_y]
    gt_x = [str(x) for x in gt_x]
            
    # gt文字列を作成
    gt_string = []
    for x, y in zip(gt_x, gt_y):
        gt_string.append(f"{x} | {y}")
    gt_string = "<0x0A>".join(gt_string)
    gt_string += processor.tokenizer.eos_token
    
    # if idx % 1000 == 0:
    #     print(gt_string)
    #     print(min_y, max_y, min_x, max_x)
    #     print(min(x_values), max(x_values))
    #     plt.show()
    
    plt.close()
    #plt.show()
    
    
    df = pd.DataFrame({"id": idx, "ground_truth": gt_string, "fold": -2, "x": [gt_x], "y": [gt_y], "source": "synthesis", "chart-type": chart_type, "image_path":path[3:]})
    gc.collect()
    return df

In [15]:
seed_everything(42)
from multiprocessing import Pool
import multiprocessing
num_cores = multiprocessing.cpu_count()
def _generate_graph(idx):
    return generate_graph(data_gen,idx)

num = 40000
args = list(range(num))
run = _generate_graph

with Pool(processes=num_cores) as pool:
    imap = pool.imap_unordered(run,args)
    df_list = list(tqdm(imap, total=len(args)))
df = pd.concat(df_list).reset_index(drop=True)
df.to_csv(f"./../input/benetech-synthesis-4th/graph_synthesis_{chart_type}_{num}_ver{version}.csv", index=False)

100%|██████████| 40000/40000 [38:46<00:00, 17.20it/s]
